In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [5]:
X_train

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,Maruti Swift Dzire AMT VDI,2018,50000,Diesel,Individual,Automatic,First Owner
2290,Honda City 1.5 V AT,2008,70000,Petrol,Individual,Automatic,First Owner
874,Maruti Alto 800 LXI,2017,50000,Petrol,Individual,Manual,First Owner
1907,Datsun GO T Petrol,2015,92198,Petrol,Dealer,Manual,Second Owner
3244,Hyundai EON Era Plus,2013,3240,Petrol,Individual,Manual,Second Owner
...,...,...,...,...,...,...,...
2895,Maruti Zen Estilo LXI BSIII,2008,40000,Petrol,Individual,Manual,First Owner
2763,Hyundai Grand i10 1.2 Kappa Asta,2019,15000,Petrol,Individual,Manual,First Owner
905,Maruti Ertiga VXI,2015,11918,Petrol,Trustmark Dealer,Manual,First Owner
3980,Hyundai Creta 1.6 CRDi SX Option,2015,90000,Diesel,Individual,Manual,Second Owner


__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [6]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [17]:
def func1(x):
    return np.sum(x) / x.size + 0.006 * np.random.normal(loc = 0.0, scale = 1.0, size =1)[0]

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric

    def fit(self, X, y):

        X['y'] = y

        arr = []

        for i in self.categorical:

            temp = X.groupby(i).agg({'y':[func1]}).reset_index()
            arr.append((list(temp[i]), list(temp['y']['func1'])))


        self.arr = arr

        return self

    def transform(self, df):

        arr = self.arr

        temp = pd.DataFrame()

        c = 0

        for i in self.categorical:

            setik = set(df[i].unique())
            setik.difference_update(set(arr[c][0]))

            column = df[i].replace(arr[c][0], arr[c][1]).reset_index()[i]
            column = column.replace(list(setik), 0).reset_index()[i]

            temp = pd.concat([temp, column], axis=1)

            c+=1

        temp = pd.concat([df[self.numeric].reset_index(drop=True), temp], axis=1)

        return temp

    # def __init__(self, categorical, numeric):
    #     self.categorical = categorical
    #     self.numeric = numeric
    #
    # def fit(self, X, y):
    #     X_fit = X.copy()
    #     y_fit = y.copy()
    #     X_with_target = pd.concat((X_fit, y_fit), axis=1)
    #     self.dict_of_means = {col : ((X_with_target.groupby(col)["selling_price"].mean()) + np.random.standard_normal() * 0.006)
    #                           for col in self.categorical}
    #
    # def transform(self, df):
    #     self.df = df
    #     temp = self.df.copy()
    #     for col in self.categorical:
    #         temp[col] = temp[col].map(self.dict_of_means[col])
    #
    #     temp = temp.fillna(0)
    #     return temp

In [19]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head()

,km_driven,name,year,fuel,seller_type,transmission,owner
0,50000,13.483692,13.436559,13.093756,12.615863,13.771135,12.977730
1,70000,12.117029,11.903115,12.453832,12.615863,13.771135,12.977730
2,50000,12.310456,13.328864,12.453832,12.615863,12.639805,12.977730
3,92198,12.491443,13.042359,12.453832,13.152824,12.639805,12.463313
4,3240,12.390906,12.870886,12.453832,12.615863,12.639805,12.463313


In [9]:
X_test.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
2761,Ford Aspire Titanium Plus Diesel BSIV,2019,15000,Diesel,Individual,Manual,First Owner
3210,Maruti Swift Dzire LDI,2014,70000,Diesel,Individual,Manual,Second Owner
2606,Hyundai Verna 1.6 SX,2012,90000,Diesel,Individual,Manual,First Owner
1030,Chevrolet Beat LT Option,2016,41000,Petrol,Dealer,Manual,First Owner
3942,Honda City i DTEC SV,2014,71318,Diesel,Dealer,Manual,First Owner


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [10]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [11]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
import seaborn as sns
dict_error = {}
np.random.seed(1)
for depth in max_depth_list:
    pipe = Pipeline([("decision_tree", DecisionTreeRegressor(max_depth=depth))])
    pipe.fit(X_train, y_train)
    error = np.mean((pipe.predict(X_test) - y_test)**2)
    dict_error[depth] = error

print(dict_error)
fig = plt.figure()
fig.set_size_inches(16, 10)
sns.lineplot(data=dict_error)
plt.show()




ValueError: could not convert string to float: 'Maruti Swift Dzire AMT VDI'

In [ ]:
dict_error = {}

In [ ]:
dict_error

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
import seaborn as sns
for split_list in min_samples_split_list:
    print(split_list)
    pipe = Pipeline([("decision_tree", DecisionTreeRegressor(min_samples_split=split_list))])
    pipe.fit(X_train, y_train)
    error = np.mean((pipe.predict(X_test) - y_test)**2)
    dict_error[split_list] = error

print(dict_error)
fig = plt.figure()
fig.set_size_inches(16, 10)
sns.lineplot(data=dict_error)
plt.show()


In [ ]:
dict_error = {}
for impurity_decrease in min_impurity_decrease_list:
    pipe = Pipeline([("decision_tree", DecisionTreeRegressor(min_impurity_decrease=impurity_decrease))])
    pipe.fit(X_train, y_train)
    error = np.mean((pipe.predict(X_test) - y_test)**2)
    dict_error[impurity_decrease] = error

print(dict_error)
fig = plt.figure()
fig.set_size_inches(16, 10)
sns.lineplot(data=dict_error)
plt.show()

In [ ]:
np.random.seed(1)

Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)


pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),


                 ("decision_tree",
                  DecisionTreeRegressor())])
search = GridSearchCV(pipe,
                      param_grid, cv=4, scoring='neg_mean_squared_error')

search.fit(X_train, y_train)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

print(f"Качество лучшей модели на финальном тесте: {search.score(X_test, y_test)}")






Best parameter (CV score=-0.57809):
{'decision_tree__max_depth': 12, 'decision_tree__max_leaf_nodes': 100, 'decision_tree__min_impurity_decrease': 0.2, 'decision_tree__min_samples_split': 10}
Качество лучшей модели на финальном тесте: -0.5197845418970586
